In [1]:
## 导入必要的库
import argparse
import torch
import pandas as pd
import numpy as np
from gruNet import get_gru2

parser = argparse.ArgumentParser()

parser.add_argument('--gru_hid_dim', type = int, default = 5)
parser.add_argument('--num_feature', type = int, default = 3)
parser.add_argument('--n_layers', type = int, default = 1)
parser.add_argument('--dropout', type=float, default=0.)
# args = parser.parse_args()
args = parser.parse_known_args()[0]
model = get_gru2(args)
filepath="client1.csv"

def saveparm(model,filepath):
    ## 参数存入parm字典中，同时tensor类型变量转换成numpy array
    parm={}
    parmlist=[]
    parmshp=[]
    parmlen=[]
    for name,parameters in model.items():
        print(name,':',parameters.size())
        parm[name]=parameters.detach().numpy()
        parmshp.append(list(parameters.detach().numpy().shape))
        parmlen.extend(list(parameters.flatten().detach().numpy().shape))
        parmlist.extend(parameters.flatten().detach().numpy().tolist())
        parmarray=np.array(parmlist)
        n=8
        parmarray=np.around(parmarray*10**n)
    print(parmarray)
    np.savetxt(filepath,parmarray,delimiter=',')
    return parmlen

parmlen=saveparm(model.state_dict(),filepath)

gru.weight_ih_l0 : torch.Size([15, 3])
gru.weight_hh_l0 : torch.Size([15, 5])
gru.bias_ih_l0 : torch.Size([15])
gru.bias_hh_l0 : torch.Size([15])
gru.weight_ih_l0_reverse : torch.Size([15, 3])
gru.weight_hh_l0_reverse : torch.Size([15, 5])
gru.bias_ih_l0_reverse : torch.Size([15])
gru.bias_hh_l0_reverse : torch.Size([15])
fc.weight : torch.Size([1, 10])
fc.bias : torch.Size([1])
[-7.0043410e+06  3.7008750e+07  1.0037148e+07 -2.4236217e+07
  4.4290537e+07  1.1043400e+07  2.3226172e+07  2.0098925e+07
 -2.6431611e+07  3.9220542e+07 -2.2030658e+07  6.7837890e+06
 -4.4334900e+07 -1.1550042e+07  3.5541064e+07  2.6707447e+07
  2.2109771e+07 -3.5561442e+07 -9.7487090e+06 -3.6453882e+07
 -6.1631890e+06  2.1201056e+07  3.9520448e+07 -4.1887915e+07
  2.0306808e+07 -1.5664050e+07 -3.9882994e+07 -1.8605626e+07
  3.9936358e+07 -8.7054400e+06 -3.3660500e+05 -4.3107408e+07
  2.9478902e+07  3.7141901e+07  3.8784581e+07  2.8851813e+07
 -7.9070690e+06  2.2454980e+06  1.7868173e+07  3.1090122e+07
  4.2495

In [3]:
#参数读取
def loadparm(filepath,parmlen,precision,lam=1):
    
    parmarray2=np.loadtxt(filepath, delimiter=",")/10**precision
    flag=0
    counter=0
    lambda_=lam

    for name2, data in model.state_dict().items():

        print('  ')
        print(f'name: {name2}')
        print(f'data: {data}')

        update_per_layer=parmarray2[counter:counter+parmlen[flag]]*lambda_
        counter+=parmlen[flag]
        flag+=1

        update_per_layer=torch.from_numpy(update_per_layer)
        update_per_layer=update_per_layer.reshape_as(data)
        #print(dir(data))
        if data.type() != update_per_layer.type():
            data.copy_(update_per_layer.to(torch.int64))
        else:
            data.copy_(update_per_layer)

loadparm("client1.csv",parmlen,8)

  
name: gru.weight_ih_l0
data: tensor([[-0.1031, -0.3199,  0.3638],
        [ 0.3477,  0.1401,  0.1880],
        [-0.4379,  0.2772,  0.2713],
        [-0.2343,  0.0105,  0.0574],
        [-0.3311, -0.2290, -0.3074],
        [ 0.0447,  0.0243,  0.4072],
        [ 0.3653, -0.0548, -0.4214],
        [-0.2138, -0.3422, -0.0479],
        [ 0.3478,  0.3903, -0.1108],
        [-0.0891,  0.3905, -0.4202],
        [-0.3766,  0.0954,  0.2728],
        [-0.2775,  0.2100,  0.4064],
        [ 0.0159,  0.0860, -0.0763],
        [-0.4396,  0.3542,  0.1611],
        [ 0.3733,  0.3907, -0.0044]])
  
name: gru.weight_hh_l0
data: tensor([[-0.1480,  0.3356,  0.0892, -0.0869,  0.2751],
        [ 0.1911,  0.1506, -0.2310,  0.1936, -0.2712],
        [ 0.0677,  0.1660,  0.4217, -0.2966, -0.0423],
        [-0.3818,  0.0619,  0.2617,  0.0017,  0.2160],
        [-0.3610,  0.0080,  0.1606, -0.0259,  0.2860],
        [ 0.0250,  0.1357, -0.3693,  0.3561,  0.1408],
        [-0.2397, -0.4151,  0.1789,  0.0379, -0.00